---

## LSTM

In [1]:
import pandas as pd
import numpy as np
import plotly_express as px
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from useful_functions import *
file_path = "./results.json"

In [2]:
# --Tables--
series_train = pd.read_csv("series_train.csv")
series_test = pd.read_csv("series_test.csv")

In [3]:
dataset_train = series_train['sales'].values.reshape(-1, 1)
dataset_test = series_test['sales'].values.reshape(-1, 1)


scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(dataset_train)
scaled_test = scaler.transform(dataset_test)

Pour les réseaux de neurones, il est essentiel de normaliser les données avec `MinMaxScaler` afin de :
- Accélérer la convergence de l'entraînement (pour adam).
- Éviter que des caractéristiques avec des plages de valeurs très différentes n'influencent de manière disproportionnée les poids du modèle.

La mise à l'échelle Min-Max redimensionne les données dans une plage spécifiée, généralement entre 0 et 1, en utilisant la formule suivante :

$$
x_{\text{scaled}} = \frac{x - x_{\min}}{x_{\max} - x_{\min}}
$$

où :
- $x$ : la valeur initiale d'une caractéristique.
- $x_{\min}$ et $x_{\max}$ : les valeurs minimale et maximale de la caractéristique.
- $x_{\text{scaled}}$ : la valeur normalisée, comprise entre 0 et 1.

In [4]:
lag_size = 7 #h=0, on a pas de retard sur les données
X_train, y_train, X_test, y_test = [], [], [], []

# --training--
for i in range(lag_size, len(scaled_train)):
    X_train.append(scaled_train[i - lag_size:i, 0])
    y_train.append(scaled_train[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# --testing--
for i in range(lag_size, len(scaled_test)):
    X_test.append(scaled_test[i - lag_size:i, 0])
    y_test.append(scaled_test[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)

 ### Explication du reshape dans les séries temporelles

Lorsqu'on utilise des modèles séquentiels comme les LSTM ou GRU, les données doivent être formatées correctement. Les modèles attendent en entré les données sous la forme suivante :

$$
(n_{\text{samples}}, \text{timesteps}, \text{features})
$$

##### Reshape de `X_test`, $(n_{\text{samples}}, \text{lag\_size})$ --> $(n_{\text{samples}}, \text{timesteps}, \text{features})$

In [5]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [6]:
# initializing the LSTM
regressorLSTM = Sequential()
 
#Adding LSTM layers
regressorLSTM.add(LSTM(units=50,#bcp de neuronne car données pas lineaire, un peu compliqué
                       activation = 'tanh' ,#pour données symetrique c'est bien
                       return_sequences = False, 
                       input_shape = (X_train.shape[1],1)))
#Adding the output layer
regressorLSTM.add(Dense(1))
 
#Compiling the model
regressorLSTM.compile(optimizer = 'adam',
                      loss = 'mean_squared_error',
                      metrics = ["accuracy"])
 
#Fitting the model
regressorLSTM.fit(X_train,y_train,
              batch_size=32,
              epochs=50,
              validation_split=0.2
              )
regressorLSTM.summary()

Epoch 1/50
27/27 [==============================] - 2s 20ms/step - loss: 0.0147 - accuracy: 0.0012 - val_loss: 0.0095 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0046 - accuracy: 0.0012 - val_loss: 0.0106 - val_accuracy: 0.0000e+00
Epoch 3/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0044 - accuracy: 0.0012 - val_loss: 0.0099 - val_accuracy: 0.0000e+00
Epoch 4/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0043 - accuracy: 0.0012 - val_loss: 0.0091 - val_accuracy: 0.0000e+00
Epoch 5/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 0.0012 - val_loss: 0.0090 - val_accuracy: 0.0000e+00
Epoch 6/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0041 - accuracy: 0.0012 - val_loss: 0.0088 - val_accuracy: 0.0000e+00
Epoch 7/50
27/27 [==============================] - 0s 3ms/step - loss: 0.0041 - accuracy: 0.0012 - val_loss: 0.0092 - val_accuracy: 0.

Testing

In [7]:
test_loss = regressorLSTM.evaluate(X_test, y_test)
print("Test Loss:", test_loss)

train_predictions = regressorLSTM.predict(X_train)
test_predictions = regressorLSTM.predict(X_test)

# Inversion de la normalisation
train_predictions_inversed = scaler.inverse_transform(train_predictions)
test_predictions_inversed = scaler.inverse_transform(test_predictions)


series_train['train_Forecast_LSTM'] = np.nan
series_train.loc[lag_size:, 'train_Forecast_LSTM'] = train_predictions_inversed.flatten()

series_test['test_Forecast_LSTM'] = np.nan
series_test.loc[lag_size:, 'test_Forecast_LSTM'] = test_predictions_inversed.flatten()

19/19 [==============================] - 0s 1ms/step - loss: 0.0122 - accuracy: 0.0000e+00
Test Loss: [0.01221778430044651, 0.0]
19/19 [==============================] - 0s 949us/step


In [9]:
fig = px.line(series_train, x='date', y=['sales', 'train_Forecast_LSTM'],title="Predictions du train")
fig.show()

In [10]:
fig = px.line(series_test, x='date', y=['sales', 'test_Forecast_LSTM'],title="Predictions du test")
fig.show()

In [16]:
nrmse, mape, mae,r2 = calculate_metrics(series_test.loc[lag_size:, 'sales'], series_test.loc[lag_size:, 'test_Forecast_LSTM'])
print(nrmse)
print(mape)
print(mae)
print(r2)
        
save_model_results(
    file_path=file_path,
    model_name="LSTM",
    params={'units' : 50,'activation' : 'tanh'  },
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)

0.07672097392186936
4.058991238132185e+19
140457.47837407453
0.27781287618335004


In [ ]:
#series_train.to_csv("series_train",index =False, header=True)
#series_test.to_csv("series_test",index =False, header=True)